# Toolkits 활용 Agent
LangChain 프레임워크를 사용하는 가장 큰 이점은 3rd-party integration 되어 있는 다양한 기능들입니다.

그 중 Toolkits 는 다양한 도구를 통합하여 제공합니다.

아래 링크에서 다양한 Tools/Toolkits 를 확인할 수 있습니다.

참고

- [Agent Toolkits](https://api.python.langchain.com/en/latest/community/agent_toolkits.html)

- [Tools](https://python.langchain.com/docs/integrations/tools/)


In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_teddynote import logging
logging.langsmith("CH15-Agent-Toolkits")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH15-Agent-Toolkits


In [3]:
import os
if not os.path.exists("tmp"):
    os.mkdir("tmp")

### FileManagementToolkit

`FileManagementToolkit` 는 로컬 파일 관리를 위한 도구 모음입니다.

- `CopyFileTool`: 파일 복사
- `DeleteFileTool`: 파일 삭제
- `FileSearchTool`: 파일 검색
- `MoveFileTool`: 파일 이동
- `ReadFileTool`: 파일 읽기
- `WriteFileTool`: 파일 쓰기
- `ListDirectoryTool`: 디렉토리 목록 조회

**설정**

- `root_dir`: 파일 작업의 루트 디렉토리 설정 가능
- `selected_tools`: 특정 도구만 선택적으로 사용 가능

**동적 도구 생성**

- `get_tools` 메서드로 선택된 도구들의 인스턴스 생성
이 `FileManagementToolkit`은 로컬 파일 관리 작업을 자동화하거나 AI 에이전트에게 파일 조작 능력을 부여할 때 유용하게 사용할 수 있습니다. 단, 보안 측면에서 신중한 접근이 필요합니다.

In [4]:
from langchain_community.agent_toolkits import FileManagementToolkit

working_directory = "tmp"

toolkit = FileManagementToolkit(root_dir=str(working_directory))

available_tools = toolkit.get_tools() #메서드를 호출하여 사용 가능한 모든 파일 관리 도구를 가져옵니다.

print("[사용가능한 파일 관리 도구들]")
for tool in available_tools:
    print(f"- {tool.name}: {tool.description}")

[사용가능한 파일 관리 도구들]
- copy_file: Create a copy of a file in a specified location
- file_delete: Delete a file
- file_search: Recursively search for files in a subdirectory that match the regex pattern
- move_file: Move or rename a file from one location to another
- read_file: Read file from disk
- write_file: Write file to disk
- list_directory: List files and directories in a specified folder


In [9]:
FileManagementToolkit()

FileManagementToolkit(root_dir=None, selected_tools=None)

In [11]:
# 일부 도구만 뺴서 사용하기
tools = FileManagementToolkit(
    root_dir=(working_directory),
    selected_tools=["read_file", "file_delete", "write_file", "list_directory"]
).get_tools()
tools

[ReadFileTool(root_dir='tmp'),
 DeleteFileTool(root_dir='tmp'),
 WriteFileTool(root_dir='tmp'),
 ListDirectoryTool(root_dir='tmp')]

In [12]:
read_tool, delete_tool, write_tool, list_tool = tools
# 파일쓰기
write_tool.invoke({"file_path": "example.txt", "text": "Hello World!"})

'File written successfully to example.txt.'

In [14]:
# 파일 목록 조회
print(list_tool.invoke({}))

example.txt


In [15]:
# 파일 삭제
print(delete_tool.invoke({"file_path": "example.txt"}))

File deleted successfully: example.txt.


In [16]:
# 파일 목록 조회
print(list_tool.invoke({}))

No files found in directory .


In [34]:
from langchain.tools import tool
from typing import List, Dict
from langchain_teddynote.tools import GoogleNews

# 최신 뉴스를 가져오는 도구
@tool
def latest_news(k: int=5) -> List[Dict[str, str]]:
    """Look up latest news"""
    news_tool = GoogleNews()
    return news_tool.search_latest(k=k)

In [35]:
# 파일을 관리하는 도구
tools = FileManagementToolkit(
    root_dir=str(working_directory),
).get_tools()

# 최신 뉴스 검색 도구를 tools에 추가
tools.append(latest_news)

tools

[CopyFileTool(root_dir='tmp'),
 DeleteFileTool(root_dir='tmp'),
 FileSearchTool(root_dir='tmp'),
 MoveFileTool(root_dir='tmp'),
 ReadFileTool(root_dir='tmp'),
 WriteFileTool(root_dir='tmp'),
 ListDirectoryTool(root_dir='tmp'),
 StructuredTool(name='latest_news', description='Look up latest news', args_schema=<class 'langchain_core.utils.pydantic.latest_news'>, func=<function latest_news at 0x0000019FB816FD80>)]

In [36]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_teddynote.messages import AgentStreamParser

In [37]:
store = {}

# 프롬프트 생성
# 프롬프트는 에이전트에게 모델이 수행할 작업을 설명하는 텍스트를 제공합니다. (도구의 이름과 역할을 입력)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "Make sure to use the `latest_news` tool to find latest news. "
            "Make sure to use the `file_management` tool to manage files. ",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

# llm 생성
llm = ChatOpenAI(model="gpt-4o-mini")
# agent생성
agent = create_tool_calling_agent(llm, tools, prompt)
# agent executor 생성
agent_excutor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=False,
    handle_parsing_errors=True
)

In [38]:
def get_session_history(session_ids):
    if session_ids not in store:
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]

In [39]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_excutor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history"
)

agent_stream_parser = AgentStreamParser()

In [40]:
result = agent_with_chat_history.stream(
    {
        "input": "최신 뉴스 5개를 검색하고, 각 뉴스의 제목을 파일명으로 가지는 파일을 생성하고(.txt), "
        "파일의 내용은 뉴스의 내용과 url을 추가하세요. "
    },
    config={"configurable": {"session_id": "abc123"}},
)

print("Agent 실행 결과:")
for step in result:
    agent_stream_parser.process_agent_steps(step)

Agent 실행 결과:
[도구 호출]
Tool: latest_news
k: 5
Log: 
Invoking: `latest_news` with `{'k': 5}`



[관찰 내용]
Observation: [{'url': 'https://news.google.com/rss/articles/CBMidEFVX3lxTE9DSWFEUWp6SzJOcGduZEtZUU5XYldNX3EwRHNDb1VXYV9HSFNweWRSa2w1N2VkVHBuMkJ6WXY2OG5uWWg0c3FoZW9pVmFaOC1fR2VTcU5nVjRUQUhnbGdfOU1yai1kQUZidFAxTzdkNHktSFdP?oc=5', 'content': '북 “한국, 무인기로 평양에 3차례 삐라 살포…또 도발 땐 행동” - 한겨레'}, {'url': 'https://news.google.com/rss/articles/CBMiZkFVX3lxTE0zNmMxQWE3TlNWNkNsMmQ3ZjJPUi1sTmpCU0dFQnlXczNxY2VGejVWdVRWT3FCeWNyQktlRlBxaTNCNW5HcEFkTGRLUTBjNXlrOEhUNVh4bUxqM2dJREUtVG5YOEtndw?oc=5', 'content': '고자세 한동훈에 윤, 독대 요청...결단 없이 만나면 ‘독’ 된다 - 한겨레'}, {'url': 'https://news.google.com/rss/articles/CBMingFBVV95cUxQSUpnVU9rSmFjaVdiY2Q5R0p5a1FsR1V0RTcxc1RkeWZYTXB0aDUtNmlQdTJfQzFKZzFnaUtuMUs3R2FPMXRadzVSZ0FlODNLWGc5ajBoc0pTX0Z1V2s5NDFlcW1TbU1ZeVF5clBUNng0VW1UQVBwU2RXNGtjeUU2UDg4bjBKUmVVNjEwMzJKd2JOZUJZSmFiVXQwVjJ5UdIBsgFBVV95cUxPUVFVai0xSDBWTlEydFlqUWk3WWNwMUZ6MXY3enRveFRSeDV1c2JRNkpiMkpiQ3FxYnhZUXc2ZWZsRURCd

In [41]:
result = agent_with_chat_history.stream(
    {
        "input": "이전에 생성한 파일 제목 맨 앞에 제목에 어울리는 emoji를 추가하여 파일명을 변경하세요. "
        "파일명도 깔끔하게 변경하세요. "
    },
    config={"configurable": {"session_id": "abc123"}},
)

print("Agent 실행 결과:")
for step in result:
    agent_stream_parser.process_agent_steps(step)

Agent 실행 결과:
[도구 호출]
Tool: list_directory
Log: 
Invoking: `list_directory` with `{}`



[관찰 내용]
Observation: 2학기_연속_‘휴학_금지’로_학칙_개정되나…대학_총장_설득_나선_政.txt
고자세_한동훈에_윤_독대_요청...결단_없이_만나면_‘독’_된다.txt
노벨_평화상에_일본_핵무기_폐기_단체_‘니혼_히단쿄’.txt
북_한국_무인기로_평양에_3차례_삐라_살포…또_도발_땐_행동.txt
윤_대통령_동남아_순방에서_귀국…마중_나온_한동훈과_짧은_악수.txt
[도구 호출]
Tool: move_file
source_path: 북_한국_무인기로_평양에_3차례_삐라_살포…또_도발_땐_행동.txt
destination_path: 🕊️_북한_무인기_도발_뉴스.txt
Log: 
Invoking: `move_file` with `{'source_path': '북_한국_무인기로_평양에_3차례_삐라_살포…또_도발_땐_행동.txt', 'destination_path': '🕊️_북한_무인기_도발_뉴스.txt'}`



[도구 호출]
Tool: move_file
source_path: 고자세_한동훈에_윤_독대_요청...결단_없이_만나면_‘독’_된다.txt
destination_path: 🔍_한동훈_독대_요청_뉴스.txt
Log: 
Invoking: `move_file` with `{'source_path': '고자세_한동훈에_윤_독대_요청...결단_없이_만나면_‘독’_된다.txt', 'destination_path': '🔍_한동훈_독대_요청_뉴스.txt'}`



[도구 호출]
Tool: move_file
source_path: 노벨_평화상에_일본_핵무기_폐기_단체_‘니혼_히단쿄’.txt
destination_path: 🏅_노벨_평화상_일본_핵무기_단체.txt
Log: 
Invoking: `move_file` with `{'source_path': '노벨_평화상에_일본_핵무기_폐기_단체_‘니혼_히단쿄’.t

In [42]:
result = agent_with_chat_history.stream(
    {
        "input": "이전에 생성한 모든 파일을 `news` 폴더를 생성한 뒤 해당 폴더에 모든 파일을 복사하세요. "
        "내용도 동일하게 복사하세요. "
    },
    config={"configurable": {"session_id": "abc123"}},
)

print("Agent 실행 결과:")
for step in result:
    agent_stream_parser.process_agent_steps(step)

Agent 실행 결과:
[도구 호출]
Tool: list_directory
Log: 
Invoking: `list_directory` with `{}`



[관찰 내용]
Observation: ✈️_윤대통령_동남아_순방_귀국_뉴스.txt
🏅_노벨_평화상_일본_핵무기_단체.txt
🏫_휴학_금지_학칙_개정_뉴스.txt
🔍_한동훈_독대_요청_뉴스.txt
🕊️_북한_무인기_도발_뉴스.txt
[도구 호출]
Tool: copy_file
source_path: 🕊️_북한_무인기_도발_뉴스.txt
destination_path: news/🕊️_북한_무인기_도발_뉴스.txt
Log: 
Invoking: `copy_file` with `{'source_path': '🕊️_북한_무인기_도발_뉴스.txt', 'destination_path': 'news/🕊️_북한_무인기_도발_뉴스.txt'}`



[관찰 내용]
Observation: Error: [Errno 2] No such file or directory: 'C:\\Users\\skyop\\JaehoNote\\15.Agent\\tmp\\news\\🕊️_북한_무인기_도발_뉴스.txt'
[도구 호출]
Tool: create_directory
dir_path: news
Log: 
Invoking: `create_directory` with `{'dir_path': 'news'}`



[도구 호출]
Tool: copy_file
source_path: 🕊️_북한_무인기_도발_뉴스.txt
destination_path: news/🕊️_북한_무인기_도발_뉴스.txt
Log: 
Invoking: `copy_file` with `{'source_path': '🕊️_북한_무인기_도발_뉴스.txt', 'destination_path': 'news/🕊️_북한_무인기_도발_뉴스.txt'}`



[도구 호출]
Tool: copy_file
source_path: 🔍_한동훈_독대_요청_뉴스.txt
destination_path: news/🔍_한동훈

In [43]:
result = agent_with_chat_history.stream(
    {"input": "news 폴더를 제외한 모든 .txt 파일을 삭제하세요."},
    config={"configurable": {"session_id": "abc123"}},
)

print("Agent 실행 결과:")
for step in result:
    agent_stream_parser.process_agent_steps(step)

Agent 실행 결과:
[도구 호출]
Tool: file_search
dir_path: .
pattern: *.txt
Log: 
Invoking: `file_search` with `{'dir_path': '.', 'pattern': '*.txt'}`



[관찰 내용]
Observation: ✈️_윤대통령_동남아_순방_귀국_뉴스.txt
🏅_노벨_평화상_일본_핵무기_단체.txt
🏫_휴학_금지_학칙_개정_뉴스.txt
🔍_한동훈_독대_요청_뉴스.txt
🕊️_북한_무인기_도발_뉴스.txt
news\✈️_윤대통령_동남아_순방_귀국_뉴스.txt
news\🏅_노벨_평화상_일본_핵무기_단체.txt
news\🏫_휴학_금지_학칙_개정_뉴스.txt
news\🔍_한동훈_독대_요청_뉴스.txt
news\🕊️_북한_무인기_도발_뉴스.txt
[도구 호출]
Tool: file_delete
file_path: ✈️_윤대통령_동남아_순방_귀국_뉴스.txt
Log: 
Invoking: `file_delete` with `{'file_path': '✈️_윤대통령_동남아_순방_귀국_뉴스.txt'}`



[도구 호출]
Tool: file_delete
file_path: 🏅_노벨_평화상_일본_핵무기_단체.txt
Log: 
Invoking: `file_delete` with `{'file_path': '🏅_노벨_평화상_일본_핵무기_단체.txt'}`



[도구 호출]
Tool: file_delete
file_path: 🏫_휴학_금지_학칙_개정_뉴스.txt
Log: 
Invoking: `file_delete` with `{'file_path': '🏫_휴학_금지_학칙_개정_뉴스.txt'}`



[도구 호출]
Tool: file_delete
file_path: 🔍_한동훈_독대_요청_뉴스.txt
Log: 
Invoking: `file_delete` with `{'file_path': '🔍_한동훈_독대_요청_뉴스.txt'}`



[도구 호출]
Tool: file_delete
file_path: 🕊️_북한